In [2]:
import numpy as np
import sympy as sym

Pi = sym.symbols('pi')
n0 = 0.153
hc = 197.326980 # MeV fm

In [3]:
class eos:
    def __init__(self, g_sigma, g_omega, b, c):
        
        self.g_sigma = g_sigma
        self.g_omega = g_omega
        self.b = b
        self.c = c

class baryon:
    def __init__(self, mass, isospin, charge, kind, var_type, mass_eff = 0.0, num_density = 0.0,\
                 frac = 0.0, kf = 0.0, ef = 0.0, chem_pot = 0.0):
    
        # variables to be established at baryon declaration
        self.mass = mass
        self.isospin = isospin
        self.charge = charge
        self.kind = kind
        self.var_type = var_type
    
        # variables to be stored later
        self.mass_eff = mass_eff
        self.num_density = num_density
        self.frac = frac
        self.kf = kf
        self.ef = ef
        self.chem_pot = chem_pot

class lepton:
    def __init__(self, mass, charge, num_density = 0.0, frac = 0.0, var_type = 0.0, kf = 0.0, chem_pot = 0.0):
        self.mass = mass
        self.charge = charge
        self.num_density = num_density
        self.frac = frac
        self.var_type = var_type
        self.kf = kf
        self.chem_pot = chem_pot

class meson:
    def __init__(self, mass, field = 0.0):
        self.mass = mass # in MeV
        self.field = field
        
class independent_var:
    def __init__(self, var, func, tilde_chem_pot = 0.0, tilde_chem_pot_val = 0.0, num_val = 0.0, total_deriv = 0.0):
        self.var = var # var is the symbol for the variable
        self.func = func 
        
        self.tilde_chem_pot = tilde_chem_pot
        self.tilde_chem_pot_val = tilde_chem_pot_val
        
        self.num_val = num_val
        
        # total derivative of fraction wrt to nB
        self.total_deriv = total_deriv

In [4]:
rmf = eos(g_sigma = 8.784820, g_omega = 8.720086, b = 0.008628, c = -0.002433)

g_sigma, g_omega, b, c = sym.symbols('g_sigma, g_omega, b, c')

# initializing symbolic eos object
rmf_sym = eos(g_sigma, g_omega, b, c)

In [5]:
# electron
electron_sym = lepton(sym.symbols('m_e'), -1, sym.symbols('n_e'), sym.symbols('x_e'), 'Independent',\
                      sym.symbols('k_F_e'), sym.symbols('\mu_e'))
electron_num = lepton(0.510, -1)

# proton 
proton_sym = baryon(sym.symbols('m'), 1/2, 1, 'Nucleon', 'Dependent', sym.symbols('m_p^*'),\
                    sym.symbols('n_p'), sym.symbols('x_p'), sym.symbols('k_F_p'),\
                    sym.symbols('E^*_F_p'), sym.symbols('mu_p'))
proton_num = baryon(939.0, 1/2, 1, 'Nucleon', 'Dependent')

# neutron 
neutron_sym = baryon(sym.symbols('m'), -1/2, 0, 'Nucleon', 'Dependent', sym.symbols('m_n^*'),\
                    sym.symbols('n_n'), sym.symbols('x_n'), sym.symbols('k_F_n'),\
                    sym.symbols('E^*_F_n'), sym.symbols('mu_n'))
neutron_num = baryon(939.0, -1/2, 0, 'Nucleon', 'Dependent')

In [6]:
# declaring the symbolic meson objects
sigma_sym = meson(sym.symbols('m_sigma'), sym.symbols('sigma'))
omega_sym = meson(sym.symbols('m_omega'), sym.symbols('omega'))

# declaring the numeric meson objects
sigma_num = meson(550.0)
omega_num = meson(783.0)

In [7]:
# initializing independent variables
nb = independent_var(sym.symbols('n_B'), sym.Function('n_B'))
xe = independent_var(sym.symbols('x_e'), sym.Function('x_e'), sym.symbols('mu tilde_x_e'))

Practice with solving $\sigma-\omega$ Model
$$
    m_\sigma^2 \sigma + m_N b g_\sigma^3 \sigma^2 + c g_\sigma^4 \sigma^3 = g_\sigma n^s \qquad n^s = \frac{m^*}{\pi^2}\left[k_F E_F - {m^*}^2\ln\frac{k_F + E_F}{m^*}\right]\\
    m_\omega^2 \omega = g_\omega n_B
$$
with constraints
$$
    n_n + n_p = n_B\\
    n_p = n_e
$$

In [8]:
eqn1 = sigma_sym.mass**2*sigma_sym.field + neutron_sym.mass*rmf_sym.b*rmf_sym.g_sigma**3*sigma_sym.field**2 + rmf_sym.c*rmf_sym.g_sigma**4*sigma_sym.field**3 - rmf_sym.g_sigma*(sym.symbols('n_n^s') + sym.symbols('n_p^s'))
eqn1

b*g_sigma**3*m*sigma**2 + c*g_sigma**4*sigma**3 - g_sigma*(n_n^s + n_p^s) + m_sigma**2*sigma

In [26]:
ns_n = neutron_sym.mass_eff/Pi**2*(neutron_sym.kf*neutron_sym.ef - neutron_sym.mass_eff**2*sym.log((neutron_sym.kf + neutron_sym.ef)/neutron_sym.mass_eff))
ns_p = proton_sym.mass_eff/Pi**2*(proton_sym.kf*proton_sym.ef - proton_sym.mass_eff**2*sym.log((proton_sym.kf + proton_sym.ef)/proton_sym.mass_eff))

In [10]:
eqn1 = eqn1.subs([(sym.symbols('n_p^s'), ns_p), (sym.symbols('n_n^s'), ns_n)])
eqn1

b*g_sigma**3*m*sigma**2 + c*g_sigma**4*sigma**3 - g_sigma*(m_n^**(E^*_F_n*k_F_n - m_n^***2*log((E^*_F_n + k_F_n)/m_n^*))/pi**2 + m_p^**(E^*_F_p*k_F_p - m_p^***2*log((E^*_F_p + k_F_p)/m_p^*))/pi**2) + m_sigma**2*sigma

In [27]:
ef_n = sym.sqrt(kf_n**2 + (neutron_sym.mass - rmf_sym.g_sigma*sigma_sym.field)**2)
ef_p = sym.sqrt(kf_p**2 + (proton_sym.mass - rmf_sym.g_sigma*sigma_sym.field)**2)
kf_n = (3*Pi**2*sym.symbols('n_n'))**(1/3)
kf_p = (3*Pi**2*sym.symbols('n_p'))**(1/3)

In [12]:
eqn1 = eqn1.subs([(neutron_sym.kf, kf_n), (neutron_sym.ef, ef_n), (proton_sym.kf, kf_p), (proton_sym.ef, ef_p)])
eqn1 = eqn1.subs(neutron_sym.mass_eff, neutron_sym.mass - rmf_sym.g_sigma*sigma_sym.field)
eqn1 = eqn1.subs(proton_sym.mass_eff, proton_sym.mass - rmf_sym.g_sigma*sigma_sym.field)
eqn1

b*g_sigma**3*m*sigma**2 + c*g_sigma**4*sigma**3 - g_sigma*((-g_sigma*sigma + m)*(2.0800838230519*(n_n*pi**2)**0.333333333333333*sqrt((n_n*pi**2)**0.666666666666667 + 0.480749856769136*(-g_sigma*sigma + m)**2) - (-g_sigma*sigma + m)**2*log((1.44224957030741*(n_n*pi**2)**0.333333333333333 + 1.44224957030741*sqrt((n_n*pi**2)**0.666666666666667 + 0.480749856769136*(-g_sigma*sigma + m)**2))/(-g_sigma*sigma + m)))/pi**2 + (-g_sigma*sigma + m)*(2.0800838230519*(n_p*pi**2)**0.333333333333333*sqrt((n_p*pi**2)**0.666666666666667 + 0.480749856769136*(-g_sigma*sigma + m)**2) - (-g_sigma*sigma + m)**2*log((1.44224957030741*(n_p*pi**2)**0.333333333333333 + 1.44224957030741*sqrt((n_p*pi**2)**0.666666666666667 + 0.480749856769136*(-g_sigma*sigma + m)**2))/(-g_sigma*sigma + m)))/pi**2) + m_sigma**2*sigma

In [13]:
eqn1 = eqn1.subs(sym.symbols('m'), neutron_num.mass)
eqn1 = eqn1.subs([(sym.symbols('b'), rmf.b), (sym.symbols('c'), rmf.c), (rmf_sym.g_sigma, rmf.g_sigma)])
eqn1 = eqn1.subs(sigma_sym.mass, sigma_num.mass)
eqn1 = eqn1.subs(Pi, np.pi)
eqn1 = eqn1.subs([(neutron_sym.num_density, nb.var*(1-electron_sym.frac)),(proton_sym.num_density, nb.var*electron_sym.frac)])
eqn1

-14.4901732481197*sigma**3 + 5492.55393864515*sigma**2 + 302500.0*sigma - 0.890088360484882*(939.0 - 8.78482*sigma)*(2904.95399497705*(n_B*x_e)**0.333333333333333*sqrt(1.08546581068471e-5*(n_B*x_e)**0.666666666666667 + (1 - 0.009355505857295*sigma)**2) - 881721.0*(1 - 0.009355505857295*sigma)**2*log((3.09366772628014*(n_B*x_e)**0.333333333333333 + 939.0*sqrt(1.08546581068471e-5*(n_B*x_e)**0.666666666666667 + (1 - 0.009355505857295*sigma)**2))/(939.0 - 8.78482*sigma))) - 0.890088360484882*(939.0 - 8.78482*sigma)*(2904.95399497705*(n_B*(1 - x_e))**0.333333333333333*sqrt(1.08546581068471e-5*(n_B*(1 - x_e))**0.666666666666667 + (1 - 0.009355505857295*sigma)**2) - 881721.0*(1 - 0.009355505857295*sigma)**2*log((3.09366772628014*(n_B*(1 - x_e))**0.333333333333333 + 939.0*sqrt(1.08546581068471e-5*(n_B*(1 - x_e))**0.666666666666667 + (1 - 0.009355505857295*sigma)**2))/(939.0 - 8.78482*sigma)))

At this stage, the equation of motion for the sigma field has three unknowns: nB, xe, and sigma. We have a second equation that we can use: beta-equilibrium. Then, we can solve for everything in terms of nB values.

In [14]:
nB = np.arange(0.27, 8.00, 0.01)
nB_mev = nB*n0*hc**3

In [15]:
# Beta Equilibrium
=

SyntaxError: invalid syntax (<ipython-input-15-d9323f8de746>, line 2)

Solving for Particle Fractions
1. We have a few things to consider:
- Baryon number conservation
- Charge neutrality
- Beta equilibrium 
2. When considering just $npe$ matter. We have four unknowns: $n_B, n_p, n_n, n_e$. But we have three equations so we can write everything as a function of $n_B$. 
$$
    n_B = n_p + n_n\\
    n_p = n_e\\
    \mu_n = \mu_p + \mu_e
$$
3. For these equations, it might be more simple to re-write most things in terms of the first two first... 

In [16]:
eqn2 = neutron_sym.chem_pot - proton_sym.chem_pot - electron_sym.chem_pot

In [17]:
eqn2 = eqn2.subs(electron_sym.chem_pot, sym.sqrt(electron_sym.kf**2 + electron_sym.mass**2))
eqn2

mu_n - mu_p - sqrt(k_F_e**2 + m_e**2)

In [18]:
eqn2 = eqn2.subs(proton_sym.chem_pot, sym.sqrt(proton_sym.kf**2 + (proton_sym.mass - rmf_sym.g_sigma*sigma_sym.field)**2))
eqn2

mu_n - sqrt(k_F_e**2 + m_e**2) - sqrt(k_F_p**2 + (-g_sigma*sigma + m)**2)

In [19]:
eqn2 = eqn2.subs(neutron_sym.chem_pot, sym.sqrt(neutron_sym.kf**2 + (neutron_sym.mass - rmf_sym.g_sigma*sigma_sym.field)**2))
eqn2

-sqrt(k_F_e**2 + m_e**2) + sqrt(k_F_n**2 + (-g_sigma*sigma + m)**2) - sqrt(k_F_p**2 + (-g_sigma*sigma + m)**2)

In [20]:
eqn2 = eqn2.subs([(neutron_sym.kf, kf_n), (proton_sym.kf, kf_p), (electron_sym.kf, (3*Pi**2*nb.var*electron_sym.frac)**((1)/3))])
eqn2

-1.44224957030741*sqrt(0.480749856769136*m_e**2 + (n_B*pi**2*x_e)**0.666666666666667) + 1.44224957030741*sqrt((n_n*pi**2)**0.666666666666667 + 0.480749856769136*(-g_sigma*sigma + m)**2) - 1.44224957030741*sqrt((n_p*pi**2)**0.666666666666667 + 0.480749856769136*(-g_sigma*sigma + m)**2)

In [21]:
eqn2 = eqn2.subs(sym.symbols('m'), neutron_num.mass)
eqn2 = eqn2.subs(sym.symbols('m_e'), electron_num.mass)
eqn2 = eqn2.subs(sigma_sym.mass, sigma_num.mass)
eqn2 = eqn2.subs(Pi, np.pi)
eqn2 = eqn2.subs(rmf_sym.g_sigma, rmf.g_sigma)
eqn2 = eqn2.subs([(neutron_sym.num_density, nb.var*(1-electron_sym.frac)),(proton_sym.num_density, nb.var*electron_sym.frac)])
eqn2

-939.0*sqrt(1.08546581068471e-5*(n_B*x_e)**0.666666666666667 + (1 - 0.009355505857295*sigma)**2) - 3.09366772628014*sqrt((n_B*x_e)**0.666666666666667 + 0.027176468373837) + 939.0*sqrt(1.08546581068471e-5*(n_B*(1 - x_e))**0.666666666666667 + (1 - 0.009355505857295*sigma)**2)

### Equation 2 down, written in terms of our 3 variables... now we can just iterate through nB and then solve these two system of equations for the two unknowns!

In [187]:
eqn1 = sym.N(eqn1.subs(nb.var, nB_mev[2]))
eqn2 = sym.N(eqn2.subs(nb.var, nB_mev[2]))
#sym.nonlinsolve([eqn1, eqn2], [xe.var, sigma_sym.field]) 

In [188]:
eqn1

-14.4901732481197*sigma**3 + 5492.55393864515*sigma**2 + 302500.0*sigma - 0.890088360484882*(939.0 - 8.78482*sigma)*(202934.337387096*x_e**0.333333333333333*(0.0529722849471088*x_e**0.666666666666667 + (1.0 - 0.009355505857295*sigma)**2)**0.5 - 881721.0*(1.0 - 0.009355505857295*sigma)**2*log((216.117505204576*x_e**0.333333333333333 + 939.0*sqrt(0.0529722849471088*x_e**0.666666666666667 + (1 - 0.009355505857295*sigma)**2))/(939.0 - 8.78482*sigma))) - 0.890088360484882*(939.0 - 8.78482*sigma)*(-881721.0*(1.0 - 0.009355505857295*sigma)**2*log((216.117505204576*(1 - x_e)**0.333333333333333 + 939.0*sqrt((1 - 0.009355505857295*sigma)**2 + 0.0529722849471088*(1 - x_e)**0.666666666666667))/(939.0 - 8.78482*sigma)) + 202934.337387096*(1.0 - x_e)**0.333333333333333*((1.0 - 0.009355505857295*sigma)**2 + 0.0529722849471088*(1.0 - x_e)**0.666666666666667)**0.5)

In [189]:
sym.nsolve([eqn1, eqn2],[xe.var, sigma_sym.field], (1,1))

TypeError: '>=' not supported between instances of 'NoneType' and 'int'

In [ ]:
for i in range(len(nB_mev)):
    eqn1 = eqn1.subs(nb.var, nB_mev[i])
    eqn2 = eqn2.subs(nb.var, nB_mev[i])
    sym.solve([eqn1, eqn2], [xe.var, sigma_sym.field])

## Omega Field
$$
    \omega = \frac{g_\omega}{m_\omega^2}n_B
$$

In [128]:
omega = np.zeros(len(nB))
for i in range(len(nB)):
    omega[i] = rmf.g_omega/omega_num.mass**2*nB_mev[i]
omega
# results match up extremely well with data from Suprovo for this field... 
# up until lambda baryon appears so we'll have to see when we learn how to solve baryonic... 

array([  4.51452276,   4.68172731,   4.84893185,   5.0161364 ,
         5.18334094,   5.35054549,   5.51775004,   5.68495458,
         5.85215913,   6.01936368,   6.18656822,   6.35377277,
         6.52097732,   6.68818186,   6.85538641,   7.02259096,
         7.1897955 ,   7.35700005,   7.5242046 ,   7.69140914,
         7.85861369,   8.02581824,   8.19302278,   8.36022733,
         8.52743188,   8.69463642,   8.86184097,   9.02904552,
         9.19625006,   9.36345461,   9.53065916,   9.6978637 ,
         9.86506825,  10.0322728 ,  10.19947734,  10.36668189,
        10.53388644,  10.70109098,  10.86829553,  11.03550008,
        11.20270462,  11.36990917,  11.53711372,  11.70431826,
        11.87152281,  12.03872736,  12.2059319 ,  12.37313645,
        12.540341  ,  12.70754554,  12.87475009,  13.04195464,
        13.20915918,  13.37636373,  13.54356828,  13.71077282,
        13.87797737,  14.04518192,  14.21238646,  14.37959101,
        14.54679556,  14.7140001 ,  14.88120465,  15.04

We want to solve:
$$
    m_\sigma^2 \sigma + \frac{dU}{d\sigma} = \sum_i g_{\sigma i}n^s_i\\
    m_\omega^2 \omega_0 = \sum_i g_{\omega i} n_i\\
    \mu_n = \mu_p + \mu_e\\
    k_p = k_e \quad \leftrightarrow \quad n_p = n_e\\
    n_B = n_n + n_p
$$
Unknowns: $\omega, \sigma, k_p, k_e, k_n$

In [22]:
ns_n = neutron_sym.mass_eff/Pi**2*(neutron_sym.kf*neutron_sym.ef - neutron_sym.mass_eff**2*sym.log((neutron_sym.kf + neutron_sym.ef)/neutron_sym.mass_eff))

In [31]:
ns_new = ns_n.subs(neutron_sym.ef, sym.sqrt(neutron_sym.kf**2 + neutron_sym.mass_eff**2 ))
ns_new = ns_new.subs(neutron_sym.mass_eff, neutron_sym.mass - rmf_sym.g_sigma*sigma_sym.field)
ns_new

(-g_sigma*sigma + m)*(k_F_n*sqrt(k_F_n**2 + (-g_sigma*sigma + m)**2) - (-g_sigma*sigma + m)**2*log((k_F_n + sqrt(k_F_n**2 + (-g_sigma*sigma + m)**2))/(-g_sigma*sigma + m)))/pi**2